In [34]:
import pandas as pd
import pathlib

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [22]:
p = pathlib.Path('.')
path_name = {path:path.name.replace('tbl','').replace('.csv', '') for path in list(p.glob('data/*.csv'))}
tables = {path.name.replace('tbl','').replace('.csv', ''):pd.read_csv(path, index_col=f'{path_name[path]}ID') for path in path_name}

In [23]:
tables['Film']['FilmReleaseDate'] = pd.to_datetime(tables['Film']['FilmReleaseDate'])
tables['Film']['FilmBoxOfficeDollars'] = tables['Film']['FilmBoxOfficeDollars'] /10**6
tables['Film']['FilmBudgetDollars'] = tables['Film']['FilmBudgetDollars'] /10**6
tables['Film']['FilmBenefits'] = tables['Film']['FilmBoxOfficeDollars'] - tables['Film']['FilmBudgetDollars']
tables['Film']['FilmReleaseDate'] = pd.to_datetime(tables['Film']['FilmReleaseDate'])
tables['Film']['FilmReleaseYear'] = tables['Film']['FilmReleaseDate'].dt.year

In [7]:
tables['Director']

,DirectorName,DirectorDOB,DirectorGender
DirectorID,,,
4,Steven Spielberg,1946-12-18 00:00:00.000,Male
5,Joel Coen,1954-11-29 00:00:00.000,Male
6,Ethan Coen,1957-09-21 00:00:00.000,Male
7,George Lucas,1944-05-14 00:00:00.000,Male
8,Ang Lee,1954-10-23 00:00:00.000,Male
...,...,...,...
121,Merian C. Cooper,1893-10-24 00:00:00.000,Male
122,Akira Kurosawa,1910-10-23 00:00:00.000,Male
123,Morgan Spurlock,1970-11-07 00:00:00.000,Male


In [47]:
df = tables['Actor'].merge(tables['Cast'], left_index=True, right_on='CastActorID').merge(tables['Film'][['FilmName','FilmBoxOfficeDollars', 'FilmBudgetDollars', 'FilmBenefits']], left_on='CastFilmID', right_index=True)
    
df = df.groupby('ActorName').mean().sort_values('FilmBoxOfficeDollars', ascending=False)

C:\Users\nicom\AppData\Local\Temp\ipykernel_4128\4254613788.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [50]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=df.index, y=df['FilmBoxOfficeDollars'].iloc[0:10]))

fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

fig



In [16]:
tables['Actor']

,ActorName,ActorDOB,ActorGender
ActorID,,,
1,Tom Cruise,1962-07-03 09:53:21.000,Male
2,Sam Neill,1947-09-14 00:00:00.000,Male
3,Laura Dern,1967-02-10 00:00:00.000,Female
4,Jeff Goldblum,1952-10-22 00:00:00.000,Male
5,Richard Attenborough,1923-08-29 00:00:00.000,Male
...,...,...,...
337,Thomas F. Wilson,1959-04-15 00:00:00.000,Male
338,Timothy Dalton,1944-03-21 00:00:00.000,Male
339,Benicio Del Toro,1967-02-19 00:00:00.000,Male


In [20]:
df = tables['Cast'].merge(tables['Actor'],left_on='CastActorID', right_index=True)

In [21]:
df = df.merge(tables['Film'],left_on='CastFilmID', right_index=True)

In [46]:
df[df['ActorName'] == 'Keira Knightley']

,CastFilmID,CastActorID,CastCharacterName,ActorName,ActorDOB,ActorGender,FilmName,FilmReleaseDate,FilmDirectorID,FilmLanguageID,...,FilmStudioID,FilmSynopsis,FilmRunTimeMinutes,FilmCertificateID,FilmBudgetDollars,FilmBoxOfficeDollars,FilmOscarNominations,FilmOscarWins,FilmBenefits,FilmReleaseYear
CastID,,,,,,,,,,,,,,,,,,,,,
251,48,64,Sabé,Keira Knightley,1985-03-26 00:00:00.000,Female,Star Wars: Episode I - The Phantom Menace,1999-07-16,7.0,1.0,...,13.0,"The evil Trade Federation, led by Nute Gunray ...",133.0,1.0,115.0,924.288297,3.0,0.0,809.288297,1999.0
68,14,64,Elizabeth Swann,Keira Knightley,1985-03-26 00:00:00.000,Female,Pirates of the Caribbean: At World's End,2007-05-24,22.0,1.0,...,3.0,"Captain Barbossa, Will Turner and Elizabeth Sw...",168.0,4.0,150.0,952.404152,2.0,0.0,802.404152,2007.0
107,21,64,Elizabeth Swann,Keira Knightley,1985-03-26 00:00:00.000,Female,Pirates of the Caribbean: Dead Man's Chest,2006-07-06,22.0,1.0,...,3.0,Jack Sparrow races to recover the heart of Dav...,150.0,4.0,150.0,1065.659812,4.0,1.0,915.659812,2006.0
199,40,64,Elizabeth Swann,Keira Knightley,1985-03-26 00:00:00.000,Female,Pirates of the Caribbean: Curse of the Black P...,2003-08-08,22.0,1.0,...,3.0,Blacksmith Will Turner teams up with eccentric...,143.0,3.0,125.0,655.011224,5.0,0.0,530.011224,2003.0


In [43]:
actordollars = df.groupby('ActorName').aggregate(['mean','std'])[['FilmBudgetDollars','FilmBoxOfficeDollars']]
actorcount = df.groupby('ActorName').count()['FilmName']

actordollars.merge(actorcount, left_index=True,right_index=True).sort_values([('FilmBoxOfficeDollars','mean')], ascending=False).head(20)

C:\Users\nicom\AppData\Local\Temp\ipykernel_46332\506282210.py:1: FutureWarning: ['CastCharacterName', 'ActorDOB', 'ActorGender', 'FilmName', 'FilmReleaseDate', 'FilmSynopsis'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  actordollars = df.groupby('ActorName').aggregate(['mean','std'])[['FilmBudgetDollars','FilmBoxOfficeDollars']]
C:\Users\nicom\AppData\Local\Temp\ipykernel_46332\506282210.py:4: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  actordollars.merge(actorcount, left_index=True,right_index=True).sort_values([('FilmBoxOfficeDollars','mean')], ascending=False).head(20)


,"(FilmBudgetDollars, mean)","(FilmBudgetDollars, std)","(FilmBoxOfficeDollars, mean)","(FilmBoxOfficeDollars, std)",FilmName
ActorName,,,,,
Kate Winslet,200.00,NaN,1835.400000,NaN,1
Billy Zane,200.00,NaN,1835.400000,NaN,1
Bill Nighy,150.00,0.000000,1009.031982,80.083845,2
Stellan Skarsgard,150.00,0.000000,1009.031982,80.083845,2
Laura Dern,63.00,NaN,920.100000,NaN,1
Richard Attenborough,63.00,NaN,920.100000,NaN,1
Ralph Fiennes,150.00,0.000000,917.233761,30.010637,2
Keira Knightley,135.00,17.795130,899.340871,173.970354,4
Emma Watson,131.00,20.736441,897.090449,68.285695,5
